In [156]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

from catboost import CatBoostRegressor
from catboost import Pool
from catboost import cv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.base import clone
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [157]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# Setup

In [158]:
VERSION    = 11
#DIR_TRAIN  = '../input/sf-autoru-solve-v4/' # подключил к ноутбуку свой внешний датасет
#DIR_TEST   = '../input/sf-dst-car-price/'
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

# Data

In [159]:
cars = pd.read_csv('cars_new.csv')
cars.head()

,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,features,price,tax,tax_per_horse,days_in_use,mileage_per_day,years_in_use
0,0,520i,body 1,color 3,бензин,model date 6,4,1991,механическая,eng dis 2,129,300000.0,задний,Левый,Не требует ремонта,3,Оригинал,Unknown,60000.0,4515.0,35,10799,27.780350,29
1,0,316i,body 1,color 1,бензин,model date 7,4,1992,механическая,eng dis 4,102,300000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,68000.0,2550.0,25,10434,28.752156,28
2,0,318i,body 1,color 5,бензин,model date 7,4,1994,механическая,eng dis 5,115,350000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,70000.0,2875.0,25,9703,36.071318,26
3,0,520i,body 1,color 1,бензин,model date 6,4,1992,механическая,eng dis 2,150,100000.0,задний,Левый,Не требует ремонта,3,Оригинал,Unknown,70000.0,5250.0,35,10434,9.584052,28
4,0,520i,body 1,color 1,бензин,model date 6,4,1982,механическая,eng dis 2,125,250000.0,задний,Левый,Не требует ремонта,1,Оригинал,Unknown,70000.0,3125.0,25,14086,17.748119,38


In [160]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10168 entries, 0 to 10167
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   test                 10168 non-null  int64  
 1   style                10168 non-null  object 
 2   body                 10168 non-null  object 
 3   color                10168 non-null  object 
 4   fuel                 10168 non-null  object 
 5   model_date           10168 non-null  object 
 6   doors                10168 non-null  int64  
 7   production_date      10168 non-null  int64  
 8   transmission         10168 non-null  object 
 9   engine_displacement  10167 non-null  object 
 10  engine_power         10168 non-null  int64  
 11  mileage              10168 non-null  float64
 12  drive_type           10168 non-null  object 
 13  wheel                10168 non-null  object 
 14  condition            10168 non-null  object 
 15  owners               10168 non-null 

In [161]:
cars['engine_displacement']=cars['engine_displacement'].astype('str')

In [162]:
#перевод категориальных str в int, для методов которым на вход необходимо int.
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

le = LabelEncoder()

le.fit(cars['style'])
cars['style'] = le.transform(cars['style'])

le.fit(cars['body'])
cars['body'] = le.transform(cars['body'])


le.fit(cars['color'])
cars['color'] = le.transform(cars['color'])

le.fit(cars['fuel'])
cars['fuel'] = le.transform(cars['fuel'])

le.fit(cars['model_date'])
cars['model_date'] = le.transform(cars['model_date'])

le.fit(cars['transmission'])
cars['transmission'] = le.transform(cars['transmission'])

le.fit(cars['engine_displacement'])
cars['engine_displacement'] = le.transform(cars['engine_displacement'])

le.fit(cars['drive_type'])
cars['drive_type'] = le.transform(cars['drive_type'])

le.fit(cars['wheel'])
cars['wheel'] = le.transform(cars['wheel'])

le.fit(cars['condition'])
cars['condition'] = le.transform(cars['condition'])

le.fit(cars['vehicle_title'])
cars['vehicle_title'] = le.transform(cars['vehicle_title'])

In [163]:
train = cars[cars['test'] == 0]
train.reset_index(drop=True)
test = cars[cars['test'] == 1]
test.reset_index(drop=True)
sample_submission = pd.read_csv('sample_submission.csv')

# Data Preprocessing

In [164]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
       
    # ################### Feature Engineering ####################################################
    df_output['years_in_use'] = df_output['production_date'].apply(lambda x: datetime.now().year-x) #количество лет использования машины
    
    df_output['mileage_per_day'] = df_output['mileage'] / (df_output['years_in_use'] * 365) #пробег машины в день
    df_output['mileage_per_day'] = df_output.apply(lambda x: x.mileage_per_day if x.years_in_use !=0 else 0, axis=1)
    
        
    # ################### fix ############################################################## 
    # Переводим признаки из float в int (иначе catboost выдает ошибку)
    for feature in ['engine_power', 'mileage', 'production_date', 'owners','tax', 'price', 'mileage_per_day']:
        df_output[feature]=df_output[feature].astype('int32')
    
    
    
    # ################### Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
   
    df_output.drop(['features', 'test', 'features', 'condition', 'wheel', 'tax', 'years_in_use'], axis=1, inplace=True)
    
    return df_output

In [165]:
train_preproc = preproc_data(train)
X_sub = preproc_data(test)

In [166]:
X = train_preproc.drop(['price'], axis=1)
y = train_preproc.price.values
X_sub = X_sub.drop(['price'], axis=1)

In [167]:
#X = train.drop(columns=['price', 'test', 'features', 'condition', 'wheel', 'tax', 'years_in_use'])


# Train Split

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# CatBoost

# чтобы не писать весь список этих признаков, просто вывел их через nunique(). и так сойдет)
X_train.nunique()

# Keep list of all categorical features in dataset to specify this for CatBoost
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

In [169]:
# Попробовать добавить параметр text_features, указав признак style

train_data = Pool(data = X_train,
                  label = y_train,
                  cat_features =  ['style', 'body', 'color', 'fuel', 'model_date', 'doors',
                                   'transmission', 'engine_displacement', 'drive_type',
                                   'owners', 'vehicle_title', 'tax_per_horse'])

test_data = Pool(data = X_test,
                 label = y_test,
                 cat_features = ['style', 'body', 'color', 'fuel', 'model_date', 'doors',
                                   'transmission', 'engine_displacement', 'drive_type',
                                   'owners', 'vehicle_title', 'tax_per_horse'])

cv_data = Pool(data = X,
               label = y,
               cat_features = ['style', 'body', 'color', 'fuel', 'model_date', 'doors',
                                   'transmission', 'engine_displacement', 'drive_type',
                                   'owners', 'vehicle_title', 'tax_per_horse'])

# Fit

In [170]:
ctb2 = CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.005,
                         iterations=5500,
                         l2_leaf_reg=2,
                         depth=6,
                         bootstrap_type = 'Bayesian', # Bayesian Bernoulli
#                          subsample = 0.8,
                         one_hot_max_size = 5,
                         random_seed=42,
                         od_type='Iter',
                         od_wait=100)

model = ctb2.fit(train_data,
                eval_set=test_data,
                verbose_eval=1000,
                use_best_model=True,
                plot=True)
print(model)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8087363	test: 0.8601095	best: 0.8601095 (0)	total: 36.2ms	remaining: 3m 19s


1000:	learn: 0.1410138	test: 0.1445118	best: 0.1445118 (1000)	total: 22.2s	remaining: 1m 39s
2000:	learn: 0.1208969	test: 0.1300194	best: 0.1300194 (2000)	total: 44.6s	remaining: 1m 18s
3000:	learn: 0.1120029	test: 0.1273495	best: 0.1273495 (3000)	total: 1m 8s	remaining: 56.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1268280437
bestIteration = 3631

Shrink model to first 3632 iterations.


In [171]:
X_sub.reset_index(drop=True)

,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,owners,vehicle_title,tax_per_horse,days_in_use,mileage_per_day
0,87,0,0,2,2,4,2017,0,1,190,158836,0,1,1,50,1302,145
1,38,0,0,2,0,4,2019,0,1,150,10,0,1,1,35,572,0
2,114,0,0,0,4,4,2012,0,0,407,120000,2,2,1,150,3129,41
3,32,1,0,2,2,5,2015,0,0,249,111466,2,2,1,75,2033,61
4,163,1,0,2,2,5,2019,0,0,381,11891,2,1,1,150,572,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,89,0,0,0,1,4,2013,0,1,184,98000,0,3,1,50,2763,38
3833,32,1,0,2,1,5,2016,0,0,249,169615,2,1,1,75,1668,116
3834,95,0,0,0,5,4,2006,0,3,218,180000,2,3,1,65,5320,35
3835,98,0,0,0,4,4,2012,0,1,245,137500,2,3,1,75,3129,47


In [172]:
predict_data = Pool(data = X_sub,
                    cat_features = ['style', 'body', 'color', 'fuel', 'model_date', 'doors',
                                   'transmission', 'engine_displacement', 'drive_type',
                                   'owners', 'vehicle_title', 'tax_per_horse'])

# Submission

In [173]:
predict_submission = model.predict(predict_data).reshape(-1,1)
predict_submission

array([[1711948.91035034],
       [2074937.44921088],
       [1212412.47699185],
       ...,
       [ 560888.6177003 ],
       [1080286.47697682],
       [1154699.66498426]])

In [174]:
predict_submission*0.94

array([[1609231.97572932],
       [1950441.20225822],
       [1139667.72837234],
       ...,
       [ 527235.30063828],
       [1015469.28835821],
       [1085417.68508521]])

In [175]:
sample_submission = pd.read_csv('sample_submission.csv')

In [176]:
sample_submission['price'] = predict_submission
#sample_submission.to_csv('sub.csv', index=False)
display(sample_submission.head(10))

sample_submission['price'] = np.round(predict_submission, -4)
#sample_submission.to_csv('sub_round.csv', index=False)
display(sample_submission.head(10))

sample_submission['price'] = np.round(predict_submission, -4)*0.94 # 0.94 на кагле выдал результат 12.46
#sample_submission.to_csv('sub_round_coef.csv', index=False)
display(sample_submission.head(10))

,id,price
0,0,1.711949e+06
1,1,2.074937e+06
2,2,1.212412e+06
3,3,2.545752e+06
4,4,5.292265e+06
5,5,2.143541e+06
6,6,1.124489e+06
7,7,6.950656e+05
8,8,1.437899e+06
9,9,1.425245e+06


,id,price
0,0,1710000.0
1,1,2070000.0
2,2,1210000.0
3,3,2550000.0
4,4,5290000.0
5,5,2140000.0
6,6,1120000.0
7,7,700000.0
8,8,1440000.0
9,9,1430000.0


,id,price
0,0,1607400.0
1,1,1945800.0
2,2,1137400.0
3,3,2397000.0
4,4,4972600.0
5,5,2011600.0
6,6,1052800.0
7,7,658000.0
8,8,1353600.0
9,9,1344200.0


# CV

Перед тем как бежать перебирать признаки и модели, поговорим о кросс-валидации (CV)

Когда мы делаем отбор признаков или перебираем настройки модели, мы постоянно смотрим в тестовые данные (X_test), что может привести к подгону под тестовые данные. В итоге мы получим Переобучение (overfitting).
Чтобы избежать этого, можно сразу использовать кросс-валидацию по фолдам (подробнее в модуле Классический Machine Learning >Модуль 7. Валидация данных >7.2. Разбиение выборки). 

Ниже представлен Пример, как можно организовать обучение модели на 5 фолдах, с дальнейшим объединением предсказаний от каждой модели.

In [177]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(loss_function = 'MAE',
                             eval_metric = 'MAPE',
                             learning_rate=0.005,
                             iterations=5500,
                             l2_leaf_reg=2,
                             depth=6,
                             bootstrap_type = 'Bayesian', # Bayesian Bernoulli
    #                          subsample = 0.8,
                             one_hot_max_size = 5,
                             random_seed=42,
                             od_type='Iter',
                             od_wait=100)

    model.fit(X_train, y_train,
              cat_features = ['style', 'body', 'color', 'fuel', 'model_date', 'doors',
                              'transmission', 'engine_displacement', 'drive_type',
                              'owners', 'vehicle_title', 'tax_per_horse'],
              eval_set=(X_test, y_test),
              verbose_eval=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=5, shuffle=True, random_state=42).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=5,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(predict_data)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

1 Fold Test MAPE: 0.128
2 Fold Test MAPE: 0.125
3 Fold Test MAPE: 0.123
4 Fold Test MAPE: 0.137
5 Fold Test MAPE: 0.128

Mean Score: 0.128
Std Score: 0.0048
Max Score: 0.137
Min Score: 0.123


In [178]:
submissions.head(10)

,sub_1,sub_2,sub_3,sub_4,sub_5
0,1.709232e+06,1.699194e+06,1.743578e+06,1.700738e+06,1.689243e+06
1,2.164238e+06,2.172940e+06,2.239425e+06,2.316190e+06,2.337498e+06
2,1.262035e+06,1.197959e+06,1.281373e+06,1.228868e+06,1.280478e+06
3,2.533919e+06,2.548192e+06,2.564618e+06,2.600603e+06,2.579573e+06
4,5.100217e+06,5.058466e+06,5.120853e+06,5.041324e+06,4.874651e+06
5,2.064597e+06,2.062430e+06,2.115538e+06,2.098077e+06,2.057659e+06
6,1.081442e+06,1.083491e+06,1.088678e+06,1.119847e+06,1.049479e+06
7,7.053425e+05,7.143761e+05,7.044334e+05,7.291219e+05,7.090361e+05
8,1.429381e+06,1.474200e+06,1.457856e+06,1.449486e+06,1.451752e+06
9,1.409070e+06,1.406210e+06,1.438402e+06,1.411562e+06,1.431115e+06


In [179]:
sample_submission = pd.read_csv('sample_submission.csv')
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = submissions['blend'].values
sample_submission.head(10)
sample_submission.price = round(sample_submission.price/10000,0)*10000*0.92
sample_submission.to_csv(f'submission_blend_v3.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1573200.0
1,1,2070000.0
2,2,1150000.0
3,3,2364400.0
4,4,4636800.0
5,5,1913600.0
6,6,993600.0
7,7,653200.0
8,8,1334000.0
9,9,1306400.0


## Stacking

Давайте сначала разберемся, что-же такое этот Stacking.

Начнем с Ансамбля моделей:
Допустим, вы обучили Различные модели. Теперь мы можем просто объединить их предсказания и получить средневзвешенное предсказание по всем моделям. При этом, чем разнообразней модели - тем лучше результат мы получим. 

А теперь Stacking:
У нас есть предсказания от разных моделей, почему бы не использовать их как новые признаки/фитчи и не построить поверх этих предсказаний новую модель? Это основная идея Stacking-a.

Далее его еще можно бесконечно усложнять. Например, добавляя модели обученные на разных выборках и/или с разным составом признаков (bagging), или увеличивая уровни стекинга.

In [186]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')

In [195]:
#Функция compute_meta_feature принимает на вход один алгоритм и возвращает новые признаки на объектах, 
#которые не использовались во время обучения.
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features usinf the classifier cls
    
    :arg model: scikit-learn classifier
    :arg X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    
    X_meta_train = np.zeros_like(y_train, dtype = np.float32)
    X_meta_test = np.zeros(len(X_test), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
            
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
        
        print_regression_metrics(X_meta_train[predict_fold_index], y_train[predict_fold_index])
        X_meta_test += folded_clf.predict(X_test)
    
    n = cv.n_splits
    X_meta_test = X_meta_test / n
    
    return X_meta_train, X_meta_test

In [196]:
#Функция generate_meta_feature делает подобное, но принимает на вход несколько классификаторов в списке 
#и повторяет процедуру, а затем генерирует и возвращает матрицу с изначальным количеством объектов
def generate_meta_features(regressors, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(regressors)
    ]
    
    stacked_features_train = np.stack([
        features_train for features_train, features_test in features
    ], axis=-1)

    stacked_features_test = np.stack([
        features_test for features_train, features_test in features
    ], axis=-1)
    
    return stacked_features_train, stacked_features_test

In [197]:
cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return print_regression_metrics(y_test, y_test_pred)

In [198]:
from sklearn.preprocessing import StandardScaler
# Стандартизируем данные:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_sub = scaler.fit_transform(X_sub)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED),
    BaggingRegressor(ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_SEED)),
    CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.005,
                         iterations=4500,
                         l2_leaf_reg=2,
                         depth=6,
                         bootstrap_type = 'Bayesian',
                         random_seed=42,
                         od_type='Iter',
                         od_wait=100)
    ], X_train, X_test, y_train, cv)


#Строим мета-алгоритм

final_model = LinearRegression()
final_model.fit(stacked_features_train, y_train)

compute_metric(final_model, X_train=stacked_features_train, X_test=stacked_features_test, y_train=y_train)

RMSE = 329603.60, MAE = 169782.99, R-sq = 0.92, MAPE = 0.14 
RMSE = 361822.58, MAE = 170712.65, R-sq = 0.89, MAPE = 0.15 
RMSE = 350293.99, MAE = 170739.01, R-sq = 0.91, MAPE = 0.13 
RMSE = 298391.44, MAE = 150759.03, R-sq = 0.93, MAPE = 0.14 
RMSE = 250719.22, MAE = 150369.04, R-sq = 0.95, MAPE = 0.12 
RMSE = 338594.84, MAE = 166744.06, R-sq = 0.91, MAPE = 0.14 
RMSE = 344613.68, MAE = 163246.64, R-sq = 0.90, MAPE = 0.15 
RMSE = 336931.19, MAE = 161955.81, R-sq = 0.92, MAPE = 0.13 
RMSE = 281250.95, MAE = 148173.84, R-sq = 0.94, MAPE = 0.14 
RMSE = 228199.49, MAE = 143529.08, R-sq = 0.96, MAPE = 0.12 
0:	learn: 0.8140908	total: 2.61ms	remaining: 11.7s
1:	learn: 0.8106837	total: 5.24ms	remaining: 11.8s
2:	learn: 0.8071531	total: 7.47ms	remaining: 11.2s
3:	learn: 0.8038273	total: 9.8ms	remaining: 11s
4:	learn: 0.8004217	total: 12.2ms	remaining: 11s
5:	learn: 0.7971928	total: 14.6ms	remaining: 10.9s
6:	learn: 0.7940466	total: 16.9ms	remaining: 10.8s
7:	learn: 0.7906900	total: 19.1ms	rema

154:	learn: 0.4449188	total: 340ms	remaining: 9.53s
155:	learn: 0.4433442	total: 342ms	remaining: 9.53s
156:	learn: 0.4417854	total: 345ms	remaining: 9.53s
157:	learn: 0.4402047	total: 347ms	remaining: 9.53s
158:	learn: 0.4387064	total: 350ms	remaining: 9.56s
159:	learn: 0.4372158	total: 353ms	remaining: 9.57s
160:	learn: 0.4356429	total: 355ms	remaining: 9.57s
161:	learn: 0.4340835	total: 357ms	remaining: 9.57s
162:	learn: 0.4326471	total: 359ms	remaining: 9.56s
163:	learn: 0.4311803	total: 362ms	remaining: 9.56s
164:	learn: 0.4296520	total: 364ms	remaining: 9.55s
165:	learn: 0.4280211	total: 366ms	remaining: 9.55s
166:	learn: 0.4267074	total: 368ms	remaining: 9.55s
167:	learn: 0.4252471	total: 370ms	remaining: 9.54s
168:	learn: 0.4237816	total: 372ms	remaining: 9.54s
169:	learn: 0.4222913	total: 375ms	remaining: 9.54s
170:	learn: 0.4208722	total: 377ms	remaining: 9.54s
171:	learn: 0.4193893	total: 379ms	remaining: 9.53s
172:	learn: 0.4179936	total: 381ms	remaining: 9.53s
173:	learn: 

384:	learn: 0.2308881	total: 841ms	remaining: 8.98s
385:	learn: 0.2303729	total: 843ms	remaining: 8.98s
386:	learn: 0.2299215	total: 845ms	remaining: 8.98s
387:	learn: 0.2294023	total: 847ms	remaining: 8.98s
388:	learn: 0.2289330	total: 849ms	remaining: 8.97s
389:	learn: 0.2284239	total: 851ms	remaining: 8.97s
390:	learn: 0.2279018	total: 854ms	remaining: 8.97s
391:	learn: 0.2274053	total: 856ms	remaining: 8.97s
392:	learn: 0.2269012	total: 858ms	remaining: 8.97s
393:	learn: 0.2264929	total: 860ms	remaining: 8.96s
394:	learn: 0.2259645	total: 862ms	remaining: 8.96s
395:	learn: 0.2254549	total: 865ms	remaining: 8.96s
396:	learn: 0.2249952	total: 867ms	remaining: 8.96s
397:	learn: 0.2245276	total: 869ms	remaining: 8.96s
398:	learn: 0.2240359	total: 871ms	remaining: 8.96s
399:	learn: 0.2235631	total: 874ms	remaining: 8.95s
400:	learn: 0.2231331	total: 876ms	remaining: 8.95s
401:	learn: 0.2226804	total: 878ms	remaining: 8.95s
402:	learn: 0.2221861	total: 880ms	remaining: 8.95s
403:	learn: 

615:	learn: 0.1642608	total: 1.34s	remaining: 8.46s
616:	learn: 0.1641608	total: 1.34s	remaining: 8.46s
617:	learn: 0.1639916	total: 1.34s	remaining: 8.45s
618:	learn: 0.1638625	total: 1.35s	remaining: 8.45s
619:	learn: 0.1637222	total: 1.35s	remaining: 8.48s
620:	learn: 0.1635656	total: 1.36s	remaining: 8.48s
621:	learn: 0.1634229	total: 1.36s	remaining: 8.48s
622:	learn: 0.1632695	total: 1.36s	remaining: 8.48s
623:	learn: 0.1631045	total: 1.36s	remaining: 8.47s
624:	learn: 0.1629472	total: 1.37s	remaining: 8.47s
625:	learn: 0.1628262	total: 1.37s	remaining: 8.47s
626:	learn: 0.1626748	total: 1.37s	remaining: 8.47s
627:	learn: 0.1625422	total: 1.37s	remaining: 8.46s
628:	learn: 0.1624147	total: 1.38s	remaining: 8.46s
629:	learn: 0.1622979	total: 1.38s	remaining: 8.46s
630:	learn: 0.1621510	total: 1.38s	remaining: 8.46s
631:	learn: 0.1620357	total: 1.38s	remaining: 8.45s
632:	learn: 0.1619291	total: 1.38s	remaining: 8.45s
633:	learn: 0.1617929	total: 1.39s	remaining: 8.45s
634:	learn: 

847:	learn: 0.1420387	total: 1.84s	remaining: 7.95s
848:	learn: 0.1419769	total: 1.85s	remaining: 7.95s
849:	learn: 0.1419294	total: 1.85s	remaining: 7.94s
850:	learn: 0.1418602	total: 1.85s	remaining: 7.94s
851:	learn: 0.1418301	total: 1.85s	remaining: 7.94s
852:	learn: 0.1417861	total: 1.86s	remaining: 7.94s
853:	learn: 0.1417111	total: 1.86s	remaining: 7.93s
854:	learn: 0.1416500	total: 1.86s	remaining: 7.93s
855:	learn: 0.1416050	total: 1.86s	remaining: 7.93s
856:	learn: 0.1415578	total: 1.86s	remaining: 7.93s
857:	learn: 0.1415045	total: 1.87s	remaining: 7.93s
858:	learn: 0.1414331	total: 1.87s	remaining: 7.92s
859:	learn: 0.1413912	total: 1.87s	remaining: 7.92s
860:	learn: 0.1413434	total: 1.87s	remaining: 7.92s
861:	learn: 0.1412840	total: 1.88s	remaining: 7.92s
862:	learn: 0.1412499	total: 1.88s	remaining: 7.91s
863:	learn: 0.1411923	total: 1.88s	remaining: 7.91s
864:	learn: 0.1411209	total: 1.88s	remaining: 7.91s
865:	learn: 0.1410572	total: 1.88s	remaining: 7.91s
866:	learn: 

...

3505:	learn: 0.1076465	total: 7.75s	remaining: 2.2s
3506:	learn: 0.1076418	total: 7.75s	remaining: 2.19s
3507:	learn: 0.1076328	total: 7.75s	remaining: 2.19s
3508:	learn: 0.1076291	total: 7.76s	remaining: 2.19s
3509:	learn: 0.1076220	total: 7.76s	remaining: 2.19s
3510:	learn: 0.1076135	total: 7.76s	remaining: 2.19s
3511:	learn: 0.1076093	total: 7.76s	remaining: 2.18s
3512:	learn: 0.1076048	total: 7.77s	remaining: 2.18s
3513:	learn: 0.1075993	total: 7.77s	remaining: 2.18s
3514:	learn: 0.1075964	total: 7.77s	remaining: 2.18s
3515:	learn: 0.1075896	total: 7.77s	remaining: 2.17s
3516:	learn: 0.1075774	total: 7.77s	remaining: 2.17s
3517:	learn: 0.1075713	total: 7.78s	remaining: 2.17s
3518:	learn: 0.1075646	total: 7.78s	remaining: 2.17s
3519:	learn: 0.1075592	total: 7.78s	remaining: 2.17s
3520:	learn: 0.1075536	total: 7.78s	remaining: 2.16s
3521:	learn: 0.1075525	total: 7.79s	remaining: 2.16s
3522:	learn: 0.1075472	total: 7.79s	remaining: 2.16s
3523:	learn: 0.1075452	total: 7.79s	remaining: 

3733:	learn: 0.1064074	total: 8.25s	remaining: 1.69s
3734:	learn: 0.1064036	total: 8.26s	remaining: 1.69s
3735:	learn: 0.1064003	total: 8.26s	remaining: 1.69s
3736:	learn: 0.1063992	total: 8.26s	remaining: 1.69s
3737:	learn: 0.1063931	total: 8.26s	remaining: 1.68s
3738:	learn: 0.1063891	total: 8.27s	remaining: 1.68s
3739:	learn: 0.1063861	total: 8.27s	remaining: 1.68s
3740:	learn: 0.1063848	total: 8.27s	remaining: 1.68s
3741:	learn: 0.1063829	total: 8.27s	remaining: 1.68s
3742:	learn: 0.1063761	total: 8.27s	remaining: 1.67s
3743:	learn: 0.1063651	total: 8.28s	remaining: 1.67s
3744:	learn: 0.1063585	total: 8.28s	remaining: 1.67s
3745:	learn: 0.1063552	total: 8.28s	remaining: 1.67s
3746:	learn: 0.1063522	total: 8.28s	remaining: 1.66s
3747:	learn: 0.1063474	total: 8.29s	remaining: 1.66s
3748:	learn: 0.1063435	total: 8.29s	remaining: 1.66s
3749:	learn: 0.1063392	total: 8.29s	remaining: 1.66s
3750:	learn: 0.1063372	total: 8.29s	remaining: 1.66s
3751:	learn: 0.1063346	total: 8.29s	remaining:

3936:	learn: 0.1054480	total: 8.71s	remaining: 1.25s
3937:	learn: 0.1054448	total: 8.71s	remaining: 1.24s
3938:	learn: 0.1054388	total: 8.71s	remaining: 1.24s
3939:	learn: 0.1054339	total: 8.72s	remaining: 1.24s
3940:	learn: 0.1054250	total: 8.72s	remaining: 1.24s
3941:	learn: 0.1054206	total: 8.72s	remaining: 1.23s
3942:	learn: 0.1054119	total: 8.72s	remaining: 1.23s
3943:	learn: 0.1054057	total: 8.72s	remaining: 1.23s
3944:	learn: 0.1053968	total: 8.73s	remaining: 1.23s
3945:	learn: 0.1053930	total: 8.73s	remaining: 1.23s
3946:	learn: 0.1053858	total: 8.73s	remaining: 1.22s
3947:	learn: 0.1053833	total: 8.73s	remaining: 1.22s
3948:	learn: 0.1053805	total: 8.74s	remaining: 1.22s
3949:	learn: 0.1053748	total: 8.74s	remaining: 1.22s
3950:	learn: 0.1053719	total: 8.74s	remaining: 1.21s
3951:	learn: 0.1053679	total: 8.74s	remaining: 1.21s
3952:	learn: 0.1053611	total: 8.75s	remaining: 1.21s
3953:	learn: 0.1053564	total: 8.75s	remaining: 1.21s
3954:	learn: 0.1053502	total: 8.75s	remaining:

4161:	learn: 0.1043827	total: 9.21s	remaining: 748ms
4162:	learn: 0.1043782	total: 9.21s	remaining: 746ms
4163:	learn: 0.1043737	total: 9.21s	remaining: 744ms
4164:	learn: 0.1043707	total: 9.22s	remaining: 741ms
4165:	learn: 0.1043688	total: 9.22s	remaining: 739ms
4166:	learn: 0.1043660	total: 9.22s	remaining: 737ms
4167:	learn: 0.1043649	total: 9.22s	remaining: 735ms
4168:	learn: 0.1043622	total: 9.23s	remaining: 733ms
4169:	learn: 0.1043579	total: 9.23s	remaining: 730ms
4170:	learn: 0.1043565	total: 9.23s	remaining: 728ms
4171:	learn: 0.1043564	total: 9.23s	remaining: 726ms
4172:	learn: 0.1043543	total: 9.23s	remaining: 724ms
4173:	learn: 0.1043426	total: 9.24s	remaining: 721ms
4174:	learn: 0.1043376	total: 9.24s	remaining: 719ms
4175:	learn: 0.1043277	total: 9.24s	remaining: 717ms
4176:	learn: 0.1043241	total: 9.24s	remaining: 715ms
4177:	learn: 0.1043181	total: 9.25s	remaining: 713ms
4178:	learn: 0.1043116	total: 9.25s	remaining: 710ms
4179:	learn: 0.1043071	total: 9.25s	remaining:

4384:	learn: 0.1033998	total: 9.71s	remaining: 255ms
4385:	learn: 0.1033962	total: 9.71s	remaining: 252ms
4386:	learn: 0.1033919	total: 9.71s	remaining: 250ms
4387:	learn: 0.1033879	total: 9.71s	remaining: 248ms
4388:	learn: 0.1033858	total: 9.72s	remaining: 246ms
4389:	learn: 0.1033794	total: 9.72s	remaining: 244ms
4390:	learn: 0.1033790	total: 9.72s	remaining: 241ms
4391:	learn: 0.1033722	total: 9.72s	remaining: 239ms
4392:	learn: 0.1033702	total: 9.73s	remaining: 237ms
4393:	learn: 0.1033649	total: 9.73s	remaining: 235ms
4394:	learn: 0.1033580	total: 9.73s	remaining: 232ms
4395:	learn: 0.1033566	total: 9.73s	remaining: 230ms
4396:	learn: 0.1033514	total: 9.73s	remaining: 228ms
4397:	learn: 0.1033453	total: 9.74s	remaining: 226ms
4398:	learn: 0.1033424	total: 9.74s	remaining: 224ms
4399:	learn: 0.1033403	total: 9.74s	remaining: 221ms
4400:	learn: 0.1033342	total: 9.74s	remaining: 219ms
4401:	learn: 0.1033336	total: 9.75s	remaining: 217ms
4402:	learn: 0.1033317	total: 9.75s	remaining:

In [199]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED),
    BaggingRegressor(ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_SEED)),
    CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.005,
                         iterations=4500,
                         l2_leaf_reg=2,
                         depth=6,
                         bootstrap_type = 'Bayesian',
                         random_seed=42,
                         od_type='Iter',
                         od_wait=100)
    ], X_train, X_sub, y_train, cv)


y_pred = np.round((final_model.predict(stacked_features_test)/1000))*1000

sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['price'] =  y_pred
sample_submission.to_csv(f'submission_stack_v1.csv', index=True)

sample_submission.head(10)

RMSE = 329603.60, MAE = 169782.99, R-sq = 0.92, MAPE = 0.14 
RMSE = 361822.58, MAE = 170712.65, R-sq = 0.89, MAPE = 0.15 
RMSE = 350293.99, MAE = 170739.01, R-sq = 0.91, MAPE = 0.13 
RMSE = 298391.44, MAE = 150759.03, R-sq = 0.93, MAPE = 0.14 
RMSE = 250719.22, MAE = 150369.04, R-sq = 0.95, MAPE = 0.12 
RMSE = 311481.37, MAE = 163479.24, R-sq = 0.93, MAPE = 0.14 
RMSE = 362567.46, MAE = 166936.31, R-sq = 0.89, MAPE = 0.15 
RMSE = 325562.83, MAE = 158538.36, R-sq = 0.92, MAPE = 0.13 
RMSE = 285935.00, MAE = 149829.59, R-sq = 0.94, MAPE = 0.14 
RMSE = 232247.61, MAE = 143401.41, R-sq = 0.96, MAPE = 0.12 
0:	learn: 0.8140908	total: 2.56ms	remaining: 11.5s
1:	learn: 0.8106837	total: 5.23ms	remaining: 11.8s
2:	learn: 0.8071531	total: 7.64ms	remaining: 11.4s
3:	learn: 0.8038273	total: 10ms	remaining: 11.3s
4:	learn: 0.8004217	total: 12.3ms	remaining: 11.1s
5:	learn: 0.7971928	total: 14.6ms	remaining: 10.9s
6:	learn: 0.7940466	total: 16.9ms	remaining: 10.8s
7:	learn: 0.7906900	total: 19.2ms	r

152:	learn: 0.4480789	total: 333ms	remaining: 9.46s
153:	learn: 0.4464133	total: 335ms	remaining: 9.46s
154:	learn: 0.4449188	total: 337ms	remaining: 9.46s
155:	learn: 0.4433442	total: 340ms	remaining: 9.46s
156:	learn: 0.4417854	total: 342ms	remaining: 9.46s
157:	learn: 0.4402047	total: 344ms	remaining: 9.46s
158:	learn: 0.4387064	total: 347ms	remaining: 9.46s
159:	learn: 0.4372158	total: 349ms	remaining: 9.46s
160:	learn: 0.4356429	total: 351ms	remaining: 9.46s
161:	learn: 0.4340835	total: 353ms	remaining: 9.46s
162:	learn: 0.4326471	total: 356ms	remaining: 9.46s
163:	learn: 0.4311803	total: 358ms	remaining: 9.46s
164:	learn: 0.4296520	total: 360ms	remaining: 9.46s
165:	learn: 0.4280211	total: 362ms	remaining: 9.46s
166:	learn: 0.4267074	total: 364ms	remaining: 9.46s
167:	learn: 0.4252471	total: 367ms	remaining: 9.45s
168:	learn: 0.4237816	total: 369ms	remaining: 9.46s
169:	learn: 0.4222913	total: 371ms	remaining: 9.45s
170:	learn: 0.4208722	total: 373ms	remaining: 9.45s
171:	learn: 

379:	learn: 0.2333569	total: 831ms	remaining: 9.01s
380:	learn: 0.2328091	total: 833ms	remaining: 9.01s
381:	learn: 0.2323021	total: 835ms	remaining: 9s
382:	learn: 0.2319186	total: 838ms	remaining: 9s
383:	learn: 0.2313837	total: 840ms	remaining: 9s
384:	learn: 0.2308881	total: 842ms	remaining: 9s
385:	learn: 0.2303729	total: 844ms	remaining: 9s
386:	learn: 0.2299215	total: 846ms	remaining: 8.99s
387:	learn: 0.2294023	total: 848ms	remaining: 8.99s
388:	learn: 0.2289330	total: 851ms	remaining: 8.99s
389:	learn: 0.2284239	total: 853ms	remaining: 8.99s
390:	learn: 0.2279018	total: 855ms	remaining: 8.98s
391:	learn: 0.2274053	total: 857ms	remaining: 8.99s
392:	learn: 0.2269012	total: 860ms	remaining: 8.98s
393:	learn: 0.2264929	total: 862ms	remaining: 8.98s
394:	learn: 0.2259645	total: 864ms	remaining: 8.98s
395:	learn: 0.2254549	total: 866ms	remaining: 8.98s
396:	learn: 0.2249952	total: 869ms	remaining: 8.98s
397:	learn: 0.2245276	total: 871ms	remaining: 8.97s
398:	learn: 0.2240359	total

604:	learn: 0.1660862	total: 1.33s	remaining: 8.55s
605:	learn: 0.1659036	total: 1.33s	remaining: 8.55s
606:	learn: 0.1657447	total: 1.33s	remaining: 8.55s
607:	learn: 0.1655938	total: 1.33s	remaining: 8.54s
608:	learn: 0.1654198	total: 1.34s	remaining: 8.54s
609:	learn: 0.1652318	total: 1.34s	remaining: 8.54s
610:	learn: 0.1650594	total: 1.34s	remaining: 8.54s
611:	learn: 0.1648846	total: 1.34s	remaining: 8.54s
612:	learn: 0.1647445	total: 1.34s	remaining: 8.53s
613:	learn: 0.1645708	total: 1.35s	remaining: 8.53s
614:	learn: 0.1644195	total: 1.35s	remaining: 8.53s
615:	learn: 0.1642608	total: 1.35s	remaining: 8.53s
616:	learn: 0.1641608	total: 1.35s	remaining: 8.52s
617:	learn: 0.1639916	total: 1.36s	remaining: 8.52s
618:	learn: 0.1638625	total: 1.36s	remaining: 8.52s
619:	learn: 0.1637222	total: 1.36s	remaining: 8.52s
620:	learn: 0.1635656	total: 1.36s	remaining: 8.51s
621:	learn: 0.1634229	total: 1.36s	remaining: 8.51s
622:	learn: 0.1632695	total: 1.37s	remaining: 8.51s
623:	learn: 

834:	learn: 0.1428020	total: 1.83s	remaining: 8.02s
835:	learn: 0.1427295	total: 1.83s	remaining: 8.02s
836:	learn: 0.1426688	total: 1.83s	remaining: 8.02s
837:	learn: 0.1426044	total: 1.83s	remaining: 8.01s
838:	learn: 0.1425500	total: 1.83s	remaining: 8.01s
839:	learn: 0.1424945	total: 1.84s	remaining: 8.01s
840:	learn: 0.1424371	total: 1.84s	remaining: 8.01s
841:	learn: 0.1423927	total: 1.84s	remaining: 8.01s
842:	learn: 0.1423224	total: 1.84s	remaining: 8s
843:	learn: 0.1422599	total: 1.85s	remaining: 8s
844:	learn: 0.1422052	total: 1.85s	remaining: 8s
845:	learn: 0.1421470	total: 1.85s	remaining: 8s
846:	learn: 0.1420955	total: 1.85s	remaining: 8s
847:	learn: 0.1420387	total: 1.86s	remaining: 7.99s
848:	learn: 0.1419769	total: 1.86s	remaining: 8s
849:	learn: 0.1419294	total: 1.86s	remaining: 8s
850:	learn: 0.1418602	total: 1.86s	remaining: 8s
851:	learn: 0.1418301	total: 1.87s	remaining: 7.99s
852:	learn: 0.1417861	total: 1.87s	remaining: 7.99s
853:	learn: 0.1417111	total: 1.87s	r

...

3825:	learn: 0.1059759	total: 9.62s	remaining: 1.69s
3826:	learn: 0.1059720	total: 9.62s	remaining: 1.69s
3827:	learn: 0.1059627	total: 9.63s	remaining: 1.69s
3828:	learn: 0.1059562	total: 9.63s	remaining: 1.69s
3829:	learn: 0.1059513	total: 9.63s	remaining: 1.68s
3830:	learn: 0.1059485	total: 9.63s	remaining: 1.68s
3831:	learn: 0.1059431	total: 9.64s	remaining: 1.68s
3832:	learn: 0.1059367	total: 9.64s	remaining: 1.68s
3833:	learn: 0.1059294	total: 9.64s	remaining: 1.68s
3834:	learn: 0.1059253	total: 9.64s	remaining: 1.67s
3835:	learn: 0.1059171	total: 9.65s	remaining: 1.67s
3836:	learn: 0.1059098	total: 9.65s	remaining: 1.67s
3837:	learn: 0.1059162	total: 9.65s	remaining: 1.66s
3838:	learn: 0.1059128	total: 9.65s	remaining: 1.66s
3839:	learn: 0.1059080	total: 9.66s	remaining: 1.66s
3840:	learn: 0.1059063	total: 9.66s	remaining: 1.66s
3841:	learn: 0.1059014	total: 9.66s	remaining: 1.65s
3842:	learn: 0.1058990	total: 9.66s	remaining: 1.65s
3843:	learn: 0.1058952	total: 9.67s	remaining:

4023:	learn: 0.1050360	total: 10.1s	remaining: 1.19s
4024:	learn: 0.1050352	total: 10.1s	remaining: 1.19s
4025:	learn: 0.1050274	total: 10.1s	remaining: 1.19s
4026:	learn: 0.1050225	total: 10.1s	remaining: 1.19s
4027:	learn: 0.1050188	total: 10.1s	remaining: 1.18s
4028:	learn: 0.1050117	total: 10.1s	remaining: 1.18s
4029:	learn: 0.1050061	total: 10.1s	remaining: 1.18s
4030:	learn: 0.1050035	total: 10.1s	remaining: 1.18s
4031:	learn: 0.1050013	total: 10.1s	remaining: 1.17s
4032:	learn: 0.1049903	total: 10.1s	remaining: 1.17s
4033:	learn: 0.1049832	total: 10.1s	remaining: 1.17s
4034:	learn: 0.1049753	total: 10.1s	remaining: 1.17s
4035:	learn: 0.1049731	total: 10.1s	remaining: 1.16s
4036:	learn: 0.1049665	total: 10.1s	remaining: 1.16s
4037:	learn: 0.1049630	total: 10.1s	remaining: 1.16s
4038:	learn: 0.1049565	total: 10.1s	remaining: 1.16s
4039:	learn: 0.1049531	total: 10.1s	remaining: 1.15s
4040:	learn: 0.1049467	total: 10.1s	remaining: 1.15s
4041:	learn: 0.1049442	total: 10.1s	remaining:

4231:	learn: 0.1040440	total: 10.6s	remaining: 671ms
4232:	learn: 0.1040392	total: 10.6s	remaining: 668ms
4233:	learn: 0.1040338	total: 10.6s	remaining: 666ms
4234:	learn: 0.1040327	total: 10.6s	remaining: 663ms
4235:	learn: 0.1040302	total: 10.6s	remaining: 661ms
4236:	learn: 0.1040272	total: 10.6s	remaining: 658ms
4237:	learn: 0.1040163	total: 10.6s	remaining: 656ms
4238:	learn: 0.1040115	total: 10.6s	remaining: 653ms
4239:	learn: 0.1040079	total: 10.6s	remaining: 651ms
4240:	learn: 0.1040059	total: 10.6s	remaining: 648ms
4241:	learn: 0.1039997	total: 10.6s	remaining: 646ms
4242:	learn: 0.1039964	total: 10.6s	remaining: 643ms
4243:	learn: 0.1039923	total: 10.6s	remaining: 641ms
4244:	learn: 0.1039911	total: 10.6s	remaining: 638ms
4245:	learn: 0.1039884	total: 10.6s	remaining: 636ms
4246:	learn: 0.1039864	total: 10.6s	remaining: 633ms
4247:	learn: 0.1039838	total: 10.6s	remaining: 631ms
4248:	learn: 0.1039815	total: 10.6s	remaining: 628ms
4249:	learn: 0.1039760	total: 10.6s	remaining:

4446:	learn: 0.1031439	total: 11.1s	remaining: 132ms
4447:	learn: 0.1031420	total: 11.1s	remaining: 130ms
4448:	learn: 0.1031346	total: 11.1s	remaining: 127ms
4449:	learn: 0.1031261	total: 11.1s	remaining: 125ms
4450:	learn: 0.1031244	total: 11.1s	remaining: 122ms
4451:	learn: 0.1031240	total: 11.1s	remaining: 120ms
4452:	learn: 0.1031171	total: 11.1s	remaining: 117ms
4453:	learn: 0.1031155	total: 11.1s	remaining: 115ms
4454:	learn: 0.1031110	total: 11.1s	remaining: 112ms
4455:	learn: 0.1031082	total: 11.1s	remaining: 110ms
4456:	learn: 0.1031044	total: 11.1s	remaining: 107ms
4457:	learn: 0.1031009	total: 11.1s	remaining: 105ms
4458:	learn: 0.1030969	total: 11.1s	remaining: 102ms
4459:	learn: 0.1030894	total: 11.1s	remaining: 99.7ms
4460:	learn: 0.1030846	total: 11.1s	remaining: 97.2ms
4461:	learn: 0.1030827	total: 11.1s	remaining: 94.7ms
4462:	learn: 0.1030766	total: 11.1s	remaining: 92.2ms
4463:	learn: 0.1030688	total: 11.1s	remaining: 89.8ms
4464:	learn: 0.1030666	total: 11.1s	remai

,id,price
0,0,1458000.0
1,1,2274000.0
2,2,1020000.0
3,3,2088000.0
4,4,4838000.0
5,5,1576000.0
6,6,992000.0
7,7,631000.0
8,8,1210000.0
9,9,1336000.0
